In [1]:
import multiprocessing
import locale
import os
import time
from random import randint

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import matplotlib.pyplot as plt
import numpy as np
import pyreadr

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import trange, tqdm

from models.autoencoder import Autoencoder
from data.PERMAD_data_reader import PermadData

locale.getpreferredencoding = lambda: "UTF-8"

# Use GPU: Send Tensors .to(device)
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

print("Device: ", device)

Device:  cpu


In [2]:
data_subjects = PermadData().data_subjects

In [3]:
idx_train, idx_test = train_test_split(range(0,len(data_subjects)), test_size=10)
print("Train :" + str(len(idx_train)))
print("Test  :" + str(len(idx_test)))

Train :31
Test  :10


In [4]:
class TrainingDS(Dataset):
  def __init__(self, data_subjects, idx_train):
    data = [data_subjects[i] for i in idx_train]  
    self.healthy = [subject[:,0] for subject in data]
    self.sick = [subject[:,-1] for subject in data]
      
  def __len__(self):
    return(len(self.healthy))

  def __getitem__(self, idx):
    return [self.healthy[idx], self.sick[idx]]

train_ds = TrainingDS(data_subjects, idx_train)
valid_ds = TrainingDS(data_subjects, idx_test)
train_iterator = DataLoader(dataset=train_ds, batch_size=1, shuffle=True)
valid_iterator = DataLoader(dataset=valid_ds, batch_size=1, shuffle=False)

In [5]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


def train(model, iterator, optimizer, criterion, device, type="h"):
    epoch_loss = 0
    model.train()

    #for h,s in tqdm(iterator, desc="Training", leave=False):
    for h,s in iterator:
        
        if type=="h":
            x = h.to(device).to(torch.float)
        else:
            x = s.to(device).to(torch.float)
        
        pred = model(x)
        loss = criterion(pred, x)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [24]:
def evaluate(model, iterator, criterion, device, type="h"):
    epoch_loss = 0
    model.eval()

    with torch.no_grad():

        #for h,s in tqdm(iterator, desc="Evaluating", leave=False):
        for (i, (h,s)) in enumerate(iterator):
            
            if type=="h":
                x = h.to(device).to(torch.float)
            else:
                x = s.to(device).to(torch.float)

            pred = model(x)
            #if i==5 or i==7:
            #    print("Encoder: ", model.encoder(x))
            loss = criterion(pred, x)
            
            epoch_loss += loss.item()
            
    return epoch_loss / len(iterator)

In [7]:
def train_loop(model, train_iterator, valid_iterator, EPOCHS=5, type="h", silent=True, schedular=None, optimizer=None):
  best_valid_loss = float('inf')
  best_train_loss = float('inf')

  for epoch in trange(EPOCHS):
      start_time = time.monotonic()

      train_loss = train(model, train_iterator, optimizer, criterion, device, type=type)
      schedular.step()
      valid_loss = evaluate(model, valid_iterator, criterion, device, type=type)
      
      if train_loss < best_train_loss:
          best_train_loss = train_loss

      if valid_loss < best_valid_loss:
          best_valid_loss = valid_loss
          if type=="h":
            torch.save(model.state_dict(), 'models/meta_parameters/autoencoder_param_healthy.pt')
          else:
            torch.save(model.state_dict(), 'models/meta_parameters/autoencoder_param_sick.pt')
            
      end_time = time.monotonic()

      epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
      if not silent:
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.6f} | Val. Loss: {valid_loss:.6f}')
  return best_train_loss, best_valid_loss

In [25]:
model_h = Autoencoder(features=98, layers=[16,4], dropout=0.0).to(device)
model_s = Autoencoder(features=98, layers=[16,4], dropout=0.0).to(device)
epochs = 100

optimizer_h = torch.optim.Adam(model_h.parameters(), lr=0.003)
schedular_h = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_h, T_max = epochs*len(train_ds))
optimizer_s = torch.optim.Adam(model_s.parameters(), lr=0.0003)
schedular_s = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_s, T_max = epochs*len(train_ds))
#criterion = nn.MSELoss(reduction="sum")
criterion = nn.MSELoss()
criterion = criterion.to(device)

best_train_loss, best_valid_loss = train_loop(model_h, train_iterator, valid_iterator, EPOCHS=200, type="h", silent=False, schedular=schedular_h, optimizer=optimizer_h)
print(f'Model h: train {best_train_loss:.5f} | valid {best_valid_loss:.5f}')

best_train_loss, best_valid_loss = train_loop(model_s, train_iterator, valid_iterator, EPOCHS=200, type="s", silent=False, schedular=schedular_s, optimizer=optimizer_s)
print(f'Model s: train {best_train_loss:.5f} | valid {best_valid_loss:.5f}')

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 0.996510 | Val. Loss: 0.984339
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 0.946828 | Val. Loss: 0.816622
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.479641 | Val. Loss: 0.213997
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.220058 | Val. Loss: 0.138704
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.148757 | Val. Loss: 0.095549
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 0.116163 | Val. Loss: 0.070758
Epoch: 07 | Epoch Time: 0m 0s
	Train Loss: 0.100950 | Val. Loss: 0.061761
Epoch: 08 | Epoch Time: 0m 0s
	Train Loss: 0.096668 | Val. Loss: 0.060864
Epoch: 09 | Epoch Time: 0m 0s
	Train Loss: 0.093053 | Val. Loss: 0.062897
Epoch: 10 | Epoch Time: 0m 0s
	Train Loss: 0.090365 | Val. Loss: 0.058832
Epoch: 11 | Epoch Time: 0m 0s
	Train Loss: 0.089493 | Val. Loss: 0.058140
Epoch: 12 | Epoch Time: 0m 0s
	Train Loss: 0.088990 | Val. Loss: 0.060085
Epoch: 13 | Epoch Time: 0m 0s
	Train Loss: 0.088595 | Val. Loss: 0.059297
Epoch: 14 | Epoch Time: 0m 0s
	Train L

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 1.015127 | Val. Loss: 1.007268
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 0.992796 | Val. Loss: 0.987289
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.973052 | Val. Loss: 0.967052
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.950036 | Val. Loss: 0.938775
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.907083 | Val. Loss: 0.869636
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 0.826814 | Val. Loss: 0.756329
Epoch: 07 | Epoch Time: 0m 0s
	Train Loss: 0.699414 | Val. Loss: 0.607526
Epoch: 08 | Epoch Time: 0m 0s
	Train Loss: 0.557451 | Val. Loss: 0.456468
Epoch: 09 | Epoch Time: 0m 0s
	Train Loss: 0.448180 | Val. Loss: 0.351551
Epoch: 10 | Epoch Time: 0m 0s
	Train Loss: 0.394237 | Val. Loss: 0.299780
Epoch: 11 | Epoch Time: 0m 0s
	Train Loss: 0.374896 | Val. Loss: 0.269354
Epoch: 12 | Epoch Time: 0m 0s
	Train Loss: 0.361366 | Val. Loss: 0.253028
Epoch: 13 | Epoch Time: 0m 0s
	Train Loss: 0.348524 | Val. Loss: 0.244921
Epoch: 14 | Epoch Time: 0m 0s
	Train L

In [9]:
h,s = valid_ds[0]
x = torch.tensor(s).to(device).to(torch.float)
x_pred = model_h(x)
sum(x_pred - x)

tensor(5.1171, grad_fn=<AddBackward0>)